In [ ]:
# import os

# # Fix to be able to import python modules inside a notebook
# os.chdir("..")

# Good code

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('max_columns', None)
# import matplotlib.pyplot as plt
# import seaborn as sns

# %matplotlib inline
# plt.style.use('ggplot')
# import datetime
# import lightgbm as lgb
# from scipy import stats
# from scipy.sparse import hstack, csr_matrix
# from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
# from nltk.corpus import stopwords
# from nltk.util import ngrams
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.preprocessing import StandardScaler
# stop = set(stopwords.words('english'))
# import os
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls
# import xgboost as xgb
# import lightgbm as lgb
# from sklearn import model_selection
# from sklearn.metrics import accuracy_score
# import json
import ast
# import eli5
# import shap
# from catboost import CatBoostRegressor
# from urllib.request import urlopen
# from PIL import Image
# from sklearn.preprocessing import LabelEncoder
# import time
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import LinearRegression
# from sklearn import linear_model

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

# Useful extensions
# %load_ext watermark
# %watermark -v -n -m -p numpy,torch,pytorch_tools,piq,realsr

%load_ext autoreload
%autoreload 2

# Nice plot formatting
%matplotlib inline

In [ ]:
!ls

**Как передавать параметры?**

0. CAPS_TEXT
1. argparse
2. click (модный argparse)
3. Hydra (для сложных конфигов)

In [ ]:
DATA_PATH = "data/train.csv"
TEST_DATA_PATH = "data/test.csv"

In [ ]:
# from train_utils import clean_df, preprocess_df, train_val_test_split, train_on

In [ ]:
train = pd.read_csv(DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)

In [ ]:
# df.head()

In [ ]:
# eval(df[~df['belongs_to_collection'].isna()]['belongs_to_collection'][0])

In [ ]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
train = text_to_dict(train)
test = text_to_dict(test)

In [ ]:
(3000 + 4500) * 8 


In [ ]:
train.sample()

In [ ]:
for i, e in enumerate(train['belongs_to_collection'][:5]):
    print(i, e)
    
    
train['collection_name'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
train['has_collection'] = train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

test['collection_name'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
test['has_collection'] = test['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

train = train.drop(['belongs_to_collection'], axis=1)
test = test.drop(['belongs_to_collection'], axis=1)

In [ ]:
for i, e in enumerate(train['genres'][:5]):
    print(i, e)

In [ ]:
print('Number of genres in films')
train['genres'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
list_of_genres = list(
    train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

In [ ]:
plt.figure(figsize = (12, 8))
text = ' '.join([i for j in list_of_genres for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title('Top genres')
plt.axis("off")
plt.show()

In [ ]:
train['num_genres'] = train['genres'].apply(lambda x: len(x) if x != {} else 0)
train['all_genres'] = train['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_genres = [m[0] for m in Counter([i for j in list_of_genres for i in j]).most_common(15)]
for g in top_genres:
    train['genre_' + g] = train['all_genres'].apply(lambda x: 1 if g in x else 0)
    
test['num_genres'] = test['genres'].apply(lambda x: len(x) if x != {} else 0)
test['all_genres'] = test['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_genres:
    test['genre_' + g] = test['all_genres'].apply(lambda x: 1 if g in x else 0)

train = train.drop(['genres'], axis=1)
test = test.drop(['genres'], axis=1)

In [ ]:
for i, e in enumerate(train['production_companies'][:5]):
    print(i, e)

In [ ]:
print('Number of production companies in films')
train['production_companies'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
train[train['production_companies'].apply(lambda x: len(x) if x != {} else 0) > 11]

In [ ]:
list_of_companies = list(train['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

In [ ]:
train['num_companies'] = train['production_companies'].apply(lambda x: len(x) if x != {} else 0)
train['all_production_companies'] = train['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_companies = [m[0] for m in Counter([i for j in list_of_companies for i in j]).most_common(30)]
for g in top_companies:
    train['production_company_' + g] = train['all_production_companies'].apply(lambda x: 1 if g in x else 0)
    
test['num_companies'] = test['production_companies'].apply(lambda x: len(x) if x != {} else 0)
test['all_production_companies'] = test['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_companies:
    test['production_company_' + g] = test['all_production_companies'].apply(lambda x: 1 if g in x else 0)

train = train.drop(['production_companies', 'all_production_companies'], axis=1)
test = test.drop(['production_companies', 'all_production_companies'], axis=1)

In [ ]:
print('Number of production countries in films')
train['production_countries'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
list_of_countries = list(train['production_countries'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_countries for i in j]).most_common(25)

In [ ]:
train['num_countries'] = train['production_countries'].apply(lambda x: len(x) if x != {} else 0)
train['all_countries'] = train['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
top_countries = [m[0] for m in Counter([i for j in list_of_countries for i in j]).most_common(25)]
for g in top_countries:
    train['production_country_' + g] = train['all_countries'].apply(lambda x: 1 if g in x else 0)
    
test['num_countries'] = test['production_countries'].apply(lambda x: len(x) if x != {} else 0)
test['all_countries'] = test['production_countries'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
for g in top_countries:
    test['production_country_' + g] = test['all_countries'].apply(lambda x: 1 if g in x else 0)

train = train.drop(['production_countries', 'all_countries'], axis=1)
test = test.drop(['production_countries', 'all_countries'], axis=1)

In [ ]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: eval(x))

In [ ]:
df["has_collection"] = df['belongs_to_collection'].apply(lambda x: 0 if x.isna() else eval(x))

In [ ]:
df[~df['belongs_to_collection'].isna()]['belongs_to_collection'].apply(lambda x: len(eval(x)))
# NaN - not a number
# NULL - empty
# .isnull()

  

In [ ]:

df['belongs_to_collection'].isna().sum()

# 2396 без коллекции
# принадлежат к колекции 

# .apply(lambda x: len(x) if x != {} else 0)#.value_counts()

In [ ]:
features = Data_df[["budget", "popularity", "revenue"]]

In [ ]:
features

In [ ]:
df = clean_df(features)

In [ ]:
df, _ = preprocess_df(df)

In [ ]:
# df = np.concatenate([Data_df[["budget"]].to_numpy(), np.ones([len(Data_df[["budget"]]), 1])], axis=1)

In [ ]:
# df

In [ ]:
df,

In [ ]:
train_df, val_df, test_df = train_val_test_split(df)

In [ ]:
def train_one_step(X_train, y_train, w, learning_rate):
    f = X_train.dot(w)
    err = f - y_train
    grad = 2 * X.T.dot(err) / len(X_train)
    w -= LR * grad
    if i % 10 == 0:
        return w, err.mean

In [ ]:
one hot encoding -
fruit = ["apple", "orange", "pear"]

fruit_apple   fruit_orange  fruit_pear
1             0             0
0             1             0
...




In [ ]:
x_val = []
y_val = []
for i in range(100):
    w, err = train_one_step(tuple(df[["budget"]]), tuple(df[["popularity"]]), w, LR)
    print(w)
    x_val.append(i)
    y_val.append(err)

In [ ]:
# df.sample(2)[['budget', ]]

# # Clean
# df = clean_df(df)

# # Split
# train_df, val_df, test_df = train_val_test_split(df)

# train_df, stats = preprocess_df(train_df)
# (val_df, _), (test_df, _) = preprocess_df(val_df, stats=stats), preprocess_df(test_df, stats=stats)

# train_df = pd.read_csv(DATA_PATH)
# # train_df['crew'].unique()
# train_df.columns
# features = train_df[["budget", "popularity", 'revenue']]
# features.columns
# features = features[(features["budget"] > 0) & (features["popularity"] > 0)]

# # features.dropna(subset=[], inplace=True)

# Garbage code

In [ ]:
def clean_df(df: pd.DataFrame):
    # Drop NaN, replace columns e.t.c 
    #  df = df[(df["budget"] > 0) & (df["popularity"] > 0)]
    df[(df["budget"] > 0) & (df["popularity"] > 0)]
    df.dropna(subset=["budget", "popularity"], inplace=True)
    return df

In [ ]:
def train_val_test_split(
    df: pd.DataFrame, train_pct: float=0.6, val_pct: float=0.2, test_pct: float=0.2):
    N = len(df)
    if train_pct + val_pct + test_pct != 1.0:
        train_pct = 0.6
        val_pct = 0.2
        test_pct = 0.2
    
    len_train = int(N*train_pct)
    len_val = int(N*val_pct)
    df.sample()
    train_df, val_df, test_df = df[:len_train], df[len_train:len_val], df[len_val:]
    return (train_df, val_df, test_df)

In [ ]:
def preprocess_df(df: pd.DataFrame, columns: tuple=('budget','popularity','revenue'), stats=None):
    # Select columns
    
    df = df[list(columns)]
    if stats is None:
        stats = {
            'mean': df.mean(),
            'std': df.std(),
        }
        # Считаем распределния и нормализуем по данным из DF
        # Standard Scaling
#          .loc[row_indexer,col_indexer] = value instead
        try:
            for i in columns:
                df[i] = ((df[i] - stats['mean'][i]) /  stats['std'][i])
                #  df.sample(10)
        except KeyError:
            return "Column not found"
        except BaseError as Err:
            return f"TraceBack: {err}"
    else:
        stats = {
            'mean': df.mean(),
            'meansquare': df.std(),
        }
    return df, stats

In [ ]:
# MSE loss function
def mse_loss(x: np.ndarray, y: np.ndarray):
    squared_error = (x - y) ** 2
    sum_squared_error = np.sum(squared_error)
    loss = sum_squared_error / val_true.size
    return loss

In [ ]:
def calculate_model_output(w, b, x):
    m = x.shape  # the number of training examples
    f_wb = np.zeros(m)
    for i in range(len(x)):
        f_wb[i] = w * x[i] + b
    return f_wb


def train_one_step(X_train, y_train, w, learning_rate):
    f = X_train.dot(w)
    err = f - y_train
    grad = 2 * X.T.dot(err) / len(X_train)
    w -= LR * grad
    if i % 10 == 0:
        x_plot.append(i)
        y_plot.append(err.mean())
        return w

def val_one_step(X_val, y_val, w):
    y_pred = X_val.dot(w)
    err = mse_loss(y_pred, y_val)
    err




In [ ]:
features.sample(10)
# standard scaling
features['budget'] = ((features.budget - features.budget.mean()) /  features.budget.std())
features['popularity'] = ((features.popularity - features.popularity.mean()) /  features.popularity.std())
features['revenue'] = ((features.revenue - features.revenue.mean()) /  features.revenue.std())
features.sample(10)

In [ ]:
features.describe()

w1 = 0.00000001
w2 = 100
w3 = 1
# 
train_x = features.to_numpy()
 
target = train_df["revenue"]
target
import numpy as np
import matplotlib.pyplot as plt

# Plotting
x_vals = np.arange(-20, 20, 0.01)
y_vals = np.square(x_vals)




In [ ]:
# TODO Do random sampling!

import random 
import numpy as np
# Fix random seed!

len_train = int(len(features) / 100 * 60)
len_val = int(len(features) / 100 * 80)
len_test = len(features) - len_train - len_val

train_features = train_x[:len1]
val_features = train_x[len1:len2]
test_features = train_x[len2:]

len(features), len(train_features), len(val_features), len(test_features)

In [ ]:
features.budget.hist(bins=20, )
features.popularity.hist(bins=20, )
features[features.popularity > 50]
features.popularity.min(), features.popularity.max()
((features.popularity - features.popularity.min()) / (features.popularity.max() - features.popularity.min())).describe()
features.popularity.describe()



In [ ]:
# Min / max scaling
((features.budget - features.budget.min()) / (features.budget.max() - features.budget.min())).describe()
# standard scaling
((features.budget - features.budget.mean()) /  features.budget.std()).describe()
train_df[train_df.popularity > 60][['original_title', 'popularity']]
features

In [ ]:
w = np.random.randn(3)

In [ ]:
plt.plot(x_vals, y_vals, "blue")
plt.plot(x_vals, y_vals, "blue")
plt.grid(True, which="major")
plt.show()
x_train = features[['budget', 'popularity']]
y_train = features['revenue'].to_numpy()
X = np.concatenate([x_train.to_numpy(), np.ones([len(x_train), 1])], axis=1)
features.to_numpy().shape, np.ones([len(features), 1]).shape
X @ w
X, y_train


In [ ]:
# TODO Переименовать переменные!

In [ ]:
x_train1 = features[['budget', 'popularity']][:2100]
y_train1 = features['revenue'].to_numpy()[:2100]
X1 = np.concatenate([x_train1.to_numpy(), np.ones([len(x_train1), 1])], axis=1)
# X1 @ w
X1, y_train1

In [ ]:
x_train2 = features[['budget', 'popularity']][2100:2600]
y_train2 = features['revenue'].to_numpy()[2100:2600]
X2 = np.concatenate([x_train2.to_numpy(), np.ones([len(x_train2), 1])], axis=1)
# X2 @ w
# X2, y_train2


x_train3 = features[['budget', 'popularity']][2601:]
y_train3 = features['revenue'].to_numpy()[2601:]
X3 = np.concatenate([x_train3.to_numpy(), np.ones([len(x_train3), 1])], axis=1)
# X3 @ w
# X3, y_train3


# w = random_normal()             # можно пробовать и другие виды инициализации
# repeat S times:                 # другой вариант: while abs(err) > tolerance
#    f = X.dot(w)                 # посчитать предсказание
#    err = f - y                  # посчитать ошибку
#    grad = 2 * X.T.dot(err) / N  # посчитать градиент
#    w -= alpha * grad            # обновить веса

In [ ]:
import func_ds
LR = 0.01
w = np.random.randn(3)
N = len(y_train)

func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(3):
    print(w)
    func_ds.train_one_step(X1, w, y_train1, N, LR, i)
        
# print(func_ds.x_plot)
# print(func_ds.y_plot)

In [ ]:


fig, ax = plt.subplots()

ax.plot(func_ds.x_plot, func_ds.y_plot)
ax.set(title='График ошибки функции на training датасете')
plt.xlabel("цикл обучения")
plt.grid(True, which="major")
plt.ylabel("квадрат ошибки")
plt.show()

In [ ]:
def change_list(a):
    x = a.pop()
    print(x)

# Side-artefacts 
A = [1, 2, 3, 4]
print(A)
change_list(A)
print(A)

In [ ]:
w = np.random.randn(3)

func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(2):
    print(w)
    func_ds.train_one_step(X2, w, y_train2, N, LR, i)

In [ ]:
fig, ax = plt.subplots()

ax.plot(func_ds.x_plot, func_ds.y_plot, "red")
ax.set(title='График ошибки функции на value датасете')
plt.grid(True, which="major")
plt.xlabel("цикл обучения")
plt.ylabel("квадрат ошибки")
plt.show()

In [ ]:
func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(1000):
    func_ds.train_one_step(X3, w, y_train3, N, LR, i)

In [ ]:
fig, ax = plt.subplots()

print(func_ds.y_plot)

ax.plot(func_ds.x_plot, func_ds.y_plot)
ax.set(title='График ошибки функции на test датасете')
plt.grid(True, which="major")
plt.xlabel("цикл обучения")
plt.ylabel("квадрат ошибки")
plt.show()